# Deep Hedging in Incomplete Markets

**MSc Thesis Experiment Runner**

Before running:
1. **Runtime > Change runtime type > A100 GPU**
2. Click **Connect**

In [ ]:
# Cell 1: Clone repo and install dependencies
!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna

import torch
print(f"\nGPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

In [ ]:
# Cell 2: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

In [ ]:
# Cell 3 (QUICK TEST): ~5 min on A100, use this first to verify everything works
!python run_experiment.py --quick

In [ ]:
# Cell 4 (FULL RUN): ~2-4 hours on A100
# Comment out Cell 3 above before running this
!python run_experiment.py \
    --paths 50000 \
    --N 200 \
    --epochs 1000 \
    --patience 15 \
    --batch_size 2048 \
    --n_trials 60 \
    --seeds 0 1 2 3 4 \
    --substeps 0 5 10

In [ ]:
# Cell 5: Preview validation plots
from IPython.display import Image, display
import glob

for img in sorted(glob.glob("outputs/plots_val/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 6: Preview 3D plots
from IPython.display import Image, display
import glob

for img in sorted(glob.glob("outputs/plots_3d/*.png")):
    print(f"\n--- {img} ---")
    display(Image(filename=img, width=700))

In [ ]:
# Cell 7: Show validation metrics
import json

with open("outputs/val_metrics.json") as f:
    metrics = json.load(f)

print(f"Best model: {metrics['best_model']}\n")
for model, agg in metrics["aggregated_val_metrics"].items():
    cvar = agg["CVaR95_shortfall"]
    mse = agg["MSE"]
    print(f"{model:6s}  CVaR95 = {cvar['mean']:.6f} +/- {cvar['std']:.6f}  "
          f"MSE = {mse['mean']:.6f} +/- {mse['std']:.6f}")

In [ ]:
# Cell 8: Download all outputs as zip
import shutil
from google.colab import files

shutil.make_archive("outputs", "zip", ".", "outputs")
files.download("outputs.zip")